# Introduction to 25 Tips in SQL, using SQL Server Database

This project explores 25 SQL Tips to enhance your understading of SQL functionalities. Each tip demonstrates the capabilities with examples that can be directly applied to the Northwind\_2023 database or easily adjusted for your database's schema. Each tip includes an explanation of its classification, why you might use it, and an example with sample output.

**Legend:**  
🗃️ Classification of the SQL function or capability.  
📝 An explanation for why or when to use the SQL tip.  
⚙️ Use case example(s).  
🚫 Any limitations or additional considerations.   
🖥️ Sample query.

## 🗃️ **Date and Time Functions**
Used to manipulate and format date and time values for time based analysis and reporting.

## **📍1. Use GETDATE to get current date and time**

📝 `GETDATE` provides the current date and time of the system, which is useful for tracking the time of an action.  
⚙️ Show all orders placed today.  

🖥️  
SELECT *  
FROM Orders  
WHERE OrderDate = CONVERT(date, GETDATE());

🚫 Since, this database has 2023 records, this function example will return no records.  
⚙️ View the current system date and time stamp.  

🖥️  
SELECT GETDATE() AS CurrentDateTime;

In [1]:
SELECT GETDATE() AS CurrentDateTime;

(1 row affected)

Total execution time: 00:00:00.085

CurrentDateTime
2024-11-02 13:26:22.460


## **📍2. Use DATEPART to extracts parts of a date**

📝 `DATEPART` extracts specific parts of a date, like year, month, or day. It’s useful for categorizing or analyzing data by time periods.    
⚙️ Show orders placed in each month.    

🖥️     
SELECT DATEPART(month, OrderDate) AS Month,  
    COUNT(OrderID) AS OrderCount  
FROM Orders  
GROUP BY DATEPART(month, OrderDate)  
ORDER BY DATEPART(month, OrderDate);  

In [5]:
SELECT DATEPART(month, OrderDate) AS Month,
    COUNT(OrderID) AS OrderCount
FROM Orders
GROUP BY DATEPART(month, OrderDate)
ORDER BY DATEPART(month, OrderDate);

(12 rows affected)

Total execution time: 00:00:00.018

Month,OrderCount
1,88
2,83
3,103
4,105
5,46
6,30
7,55
8,58
9,60
10,64


## **📍3. Use DATEDIFF to calculate date differences**

📝 `DATEDIFF` calculates the difference between two dates, which can be used to measure durations.    
⚙️ Calculate the number of days between order and shipment, for durations over 34 days.  

🖥️   
SELECT OrderID,   
    DATEDIFF(day, OrderDate, ShippedDate) AS DaysToShip  
FROM Orders  
WHERE DATEDIFF(day, OrderDate, ShippedDate)  
ORDER BY DaysToShip DESC;  

In [6]:
SELECT OrderID, 
    DATEDIFF(day, OrderDate, ShippedDate) AS DaysToShip
FROM Orders
WHERE DATEDIFF(day, OrderDate, ShippedDate) > 34
ORDER BY DaysToShip DESC;

(7 rows affected)

Total execution time: 00:00:00.014

OrderID,DaysToShip
10660,37
10777,37
10924,35
10380,35
10427,35
10545,35
10593,35


## 🗃️ **Aggregate Functions**
Perform calculations on a set of rows and return a single summarized value as the result.

## **📍4. Use MAX to find the maximum value in a column**

📝 `MAX` identifies the hishest value in in a set of values, which is useful in finding maximums or outliers.   
⚙️ Find the highest unit price in the _Products_ table.

🖥️  
SELECT ProductName,  
    UnitPrice AS MaxPrice  
FROM Products  
WHERE UnitPrice = (SELECT MAX(UnitPrice) FROM Products);  

🚫 Only works with numeric it date values. 

In [7]:
SELECT ProductName, 
    UnitPrice AS MaxPrice
FROM Products
WHERE UnitPrice = (SELECT MAX(UnitPrice) FROM Products);

(1 row affected)

Total execution time: 00:00:00.024

ProductName,MaxPrice
Côte de Blaye,263.50


## **📍5. Use MIN to find the minimum value in a column**

📝 Similarly, `MIN` identifies the lowest value in in a set of values, which is useful in finding minimums or outliers.   
⚙️ Find the minimum order quantity in the _OrderDetails_ table.

🖥️   
SELECT ProductID,   
    Quantity AS MinQuantity  
FROM OrderDetails  
WHERE Quantity = (SELECT MIN(Quantity) FROM OrderDetails);  

🚫 Only works with numeric it date values. 

In [8]:
SELECT ProductID, 
    Quantity AS MinQuantity
FROM OrderDetails
WHERE Quantity = (SELECT MIN(Quantity) FROM OrderDetails);


(17 rows affected)

Total execution time: 00:00:00.009

ProductID,MinQuantity
37,1
19,1
69,1
13,1
72,1
59,1
31,1
40,1
30,1
4,1


## **📍6. Use COUNT to count rows or non-null values.**

📝 `COUNT` is used to count the number of rows or non-null values, which can be helpful in understanding the data volume or calculating totals.   
⚙️ Find the top 5 customers with the hishest order count.

🖥️    
SELECT TOP 5 CustomerID,     
    COUNT(OrderID) AS OrderCount  
FROM Orders  
GROUP BY CustomerID  
ORDER BY OrderCount DESC;

🚫 `COUNT(column)` only counts non-null values.

In [9]:
SELECT TOP 5 CustomerID, 
    COUNT(OrderID) AS OrderCount
FROM Orders
GROUP BY CustomerID
ORDER BY OrderCount DESC;

(5 rows affected)

Total execution time: 00:00:00.012

CustomerID,OrderCount
SAVEA,31
ERNSH,30
QUICK,28
FOLKO,19
HUNGO,19


## **📍7. Use HAVING to count rows or non-null values.**

📝 `HAVING` filters after aggragation, which is useful in specifying conditions on grouped data.  
⚙️ Show customers who have placed more than 20 orders.

🖥️  
SELECT CustomerID, COUNT(OrderID) AS OrderCount  
    FROM Orders  
GROUP BY CustomerID  
HAVING COUNT(OrderID) > 20;

🚫`HAVING` is only used with `GROUP BY`, for other fitering `WHERE` can be used.

In [10]:
SELECT CustomerID, COUNT(OrderID) AS OrderCount
    FROM Orders
GROUP BY CustomerID
HAVING COUNT(OrderID) > 20;

(3 rows affected)

Total execution time: 00:00:00.009

CustomerID,OrderCount
ERNSH,30
QUICK,28
SAVEA,31


## **📍8. Use ROLLUP to calculate Subtotals and Grand Totals.**

📝 `ROLLUP` is an extension of the `GROUP BY` clause. It allows you to calculate subtotals and a grand total within a single query. Data can be summarized or grouped in levels without needing to write a query for each level of aggregation.  
⚙️ Calculate total sales by _CustomerID_ and _ProductID_, with subtotals per customer and a grand total for all sales.

🖥️  
SELECT o.CustomerID,  
    od.ProductID,  
    SUM(od.UnitPrice * od.Quantity) AS TotalSales  
FROM OrderDetails AS od  
JOIN Orders AS o ON od.OrderID = o.OrderID  
WHERE o.CustomerID IN ('ALFKI', 'ANATR')  
GROUP BY ROLLUP (o.CustomerID, od.ProductID);

🚫 `NULL` values represents the subtotals and totals, which may reduce the readability.

In [12]:
SELECT o.CustomerID, 
    od.ProductID, 
    SUM(od.UnitPrice * od.Quantity) AS TotalSales
FROM OrderDetails AS od
JOIN Orders AS o ON od.OrderID = o.OrderID
WHERE o.CustomerID IN ('ALFKI', 'ANATR')
GROUP BY ROLLUP (o.CustomerID, od.ProductID);

(24 rows affected)

Total execution time: 00:00:00.014

CustomerID,ProductID,TotalSales
ALFKI,3,60.00
ALFKI,6,400.00
ALFKI,28,775.20
ALFKI,39,378.00
ALFKI,46,24.00
ALFKI,58,530.00
ALFKI,59,825.00
ALFKI,63,878.00
ALFKI,71,430.00
ALFKI,76,270.00


## **📍9. Replace NULL values with COALESCE**

📝 `COALESCE` is a `NULL` handling function. It allows you to substitute `NULL` values with an expression that understood better by the end user. This is helpful in result sets where `NULL` values might otherwise cause confusion, like in the previous example in using `ROLLUP`.

⚙️ Calculate total sales by _CustomerID_ and _ProductID_, with subtotals per customer and a grand total for all sales with the labels to replace `NULL` values.

🖥️  
SELECT COALESCE(o.CustomerID, 'Selected Customers') AS CustomerID,  
    CASE  
        WHEN o.CustomerID IS NULL AND od.ProductID IS NULL THEN 'Grand Total'  
        ELSE COALESCE(CAST(od.ProductID AS VARCHAR), 'Subtotal')  
    END AS ProductID,  
    SUM(od.UnitPrice * od.Quantity) AS TotalSales  
FROM OrderDetails AS od  
JOIN Orders AS o ON od.OrderID = o.OrderID  
WHERE o.CustomerID IN ('ALFKI', 'ANATR')  
GROUP BY ROLLUP (o.CustomerID, od.ProductID);

🚫 Values used in `COALESCE` should match the data type of the column, else there will be an error for using integers and strings in the same column within the result set. Eg in _ProductID_, the `INT` data type is not compatible with the string _Subtotal_. So a `CAST` function is used to convert thr integer value into a `VARCHAR` string data type.

If a `INT` value was used instead to replace the values of the _ProductID_, then no conversion would be needed.  
<u>The query would then read:</u>  
SELECT COALESCE(o.CustomerID, 'Grand Total') AS CustomerID,  
    COALESCE(od.ProductID, '88888') AS ProductID, -- an integer value '8888' instead of string 'Subtotal' will not need conversion  
    SUM(od.UnitPrice * od.Quantity) AS TotalSales  
FROM OrderDetails AS od  
JOIN Orders AS o ON od.OrderID = o.OrderID  
WHERE o.CustomerID IN ('ALFKI', 'ANATR')  
GROUP BY ROLLUP (o.CustomerID, od.ProductID);

In [13]:
SELECT COALESCE(o.CustomerID, 'Selected Customers') AS CustomerID, 
    CASE 
        WHEN o.CustomerID IS NULL AND od.ProductID IS NULL THEN 'Grand Total'
        ELSE COALESCE(CAST(od.ProductID AS VARCHAR), 'Subtotal')
    END AS ProductID, 
    SUM(od.UnitPrice * od.Quantity) AS TotalSales
FROM OrderDetails AS od
JOIN Orders AS o ON od.OrderID = o.OrderID
WHERE o.CustomerID IN ('ALFKI', 'ANATR')
GROUP BY ROLLUP (o.CustomerID, od.ProductID);

(24 rows affected)

Total execution time: 00:00:00.015

CustomerID,ProductID,TotalSales
ALFKI,3,60.00
ALFKI,6,400.00
ALFKI,28,775.20
ALFKI,39,378.00
ALFKI,46,24.00
ALFKI,58,530.00
ALFKI,59,825.00
ALFKI,63,878.00
ALFKI,71,430.00
ALFKI,76,270.00


## 🗃️ **String Functions**
Used to modify text data and can help with standardizing or data cleaning.

## **📍10. Use CONCAT to combine Strings.**

📝 `CONCAT` combines multiple strings into one, which is useful for creating composite values like full names or addresses.  
⚙️ Combine _FirstName_ and _LastName_ to create full names from _Employees_ table.

🖥️    
SELECT CONCAT(FirstName, ' ', LastName) AS FullName  
FROM Employees;

🚫 `CONCAT` will return `NULL` if any argument is `NULL`.

In [14]:
SELECT CONCAT(FirstName, ' ', LastName) AS FullName
FROM Employees;

(9 rows affected)

Total execution time: 00:00:00.008

FullName
Nancy Davolio
Andrew Fuller
Janet Leverling
Margaret Peacock
Steven Buchanan
Michael Suyama
Robert King
Laura Callahan
Anne Dodsworth


## **📍11. Use UPPER to convert text to uppercase.**

📝 `UPPER` converts text into uppercase for consistent formatting.   
⚙️ Display customer names in uppercase.  

🖥️    
SELECT TOP 5 UPPER(ContactName) AS UpperCaseName  
FROM Customers;  

🚫 `Only affects alphabetic characters.

In [31]:
SELECT TOP 5 UPPER(ContactName) AS UpperCaseName
FROM Customers;

(5 rows affected)

Total execution time: 00:00:00.016

UpperCaseName
MARIA ANDERS
ANA TRUJILLO
ANTONIO MORENO
THOMAS HARDY
CHRISTINA BERGLUND


## **📍12. Use LOWER to convert text to lowercase.**

📝 `LOWER` converts text to lowercase for consisting formatting also.  
⚙️ Convert product names to lowercase.

🖥️   
SELECT TOP 5 LOWER(ProductName) AS LowerCaseProductName   
FROM Products;  

🚫 Only affects alphabetic characters.

In [15]:
SELECT TOP 5 LOWER(ProductName) AS LowerCaseProductName
FROM Products;

(5 rows affected)

Total execution time: 00:00:00.007

LowerCaseProductName
chai
chang
aniseed syrup
chef anton's cajun seasoning
chef anton's gumbo mix


## **📍13. Use REPLACE to substitute part of a String.**

📝 `REPLACE` substitutes occurrences of a specified substring with another, useful for cleaning data or standardization.  
⚙️ Replace _Ltd._ in company names with _Limited_.

🖥️   
SELECT REPLACE(CompanyName, 'Ltd.', 'Limited') AS CompanyName  
FROM Suppliers  
WHERE CompanyName LIKE '%Ltd.%';

🚫 `REPLACE` is case-sensitive.

In [16]:
SELECT REPLACE(CompanyName, 'Ltd.', 'Limited') AS CompanyName
FROM Suppliers
WHERE CompanyName LIKE '%Ltd.%';

(2 rows affected)

Total execution time: 00:00:00.007

CompanyName
"Pavlova, Limited"
"Specialty Biscuits, Limited"


## **📍14. Use TRIM to remove leading and trailing spaces.**

📝 `TRIM` removes unnecessary spaces, making it helpful for data cleaning.  
⚙️ Trim any spaces from employee names.

🖥️  
SELECT TRIM(FirstName) AS TrimFirstName   
FROM Employees;   

🚫 Only removes spaces, not other whitespace characters like tabs.

In [17]:
SELECT TRIM(FirstName) AS TrimFirstName
FROM Employees;

(9 rows affected)

Total execution time: 00:00:00.006

TrimFirstName
Nancy
Andrew
Janet
Margaret
Steven
Michael
Robert
Laura
Anne


## **📍15. Use SUBSTRING to extract part of a string.**

📝 `SUBSTRING` extracts a part of a string, which is useful for extracting specific portions like area codes from phone numbers or postal codes.  
⚙️ Extract the first 5 characters of postal codes.  

🖥️   
SELECT SUBSTRING(PostalCode, 1, 5) AS ShortPostalCode  
FROM Customers;  
  
🚫 Only returns a fixed length of characters, which may need adjusting for varying string lengths.

In [19]:
SELECT TOP 5 SUBSTRING(PostalCode, 1, 5) AS ShortPostalCode
FROM Customers;

(5 rows affected)

Total execution time: 00:00:00.011

ShortPostalCode
12209
05021
05023
WA1 1
S-958


## **📍16. Use LEFT to extract the leftmost characters of a string.**

📝 `LEFT` extracts a specified number of characters from the start of a string, useful for isolating prefixes or partial data.   
⚙️ Show the first 3 letters of each product name.  

🖥️  
SELECT TOP 5 ProductName,  
    LEFT(ProductName, 3) AS ProductPrefix  
FROM Products;

🚫 Only extracts a fixed number of characters from the left, which may need adjusting for varying string lengths.

In [22]:
SELECT TOP 5 ProductName,
	LEFT(ProductName, 3) AS ProductPrefix
FROM Products;

(5 rows affected)

Total execution time: 00:00:00.008

ProductName,ProductPrefix
Chai,Cha
Chang,Cha
Aniseed Syrup,Ani
Chef Anton's Cajun Seasoning,Che
Chef Anton's Gumbo Mix,Che


## **📍17. Use RIGHT to extract the rightmost characters of a string.**

📝 `RIGHT` extracts characters from the end of a string, useful for suffixes, codes, or last few digits.     
⚙️ Show the last 3 digits of each employee’s postal code for them to use as temporary pins.

🖥️  
SELECT EmployeeID,  
    RIGHT(PostalCode, 3) AS PostalSuffix  
FROM Employees;  

🚫 Only extracts a fixed number of characters from the right, which may need adjusting for varying string lengths.

In [23]:
SELECT EmployeeID,
	RIGHT(PostalCode, 3) AS PostalSuffix
FROM Employees;

(9 rows affected)

Total execution time: 00:00:00.007

EmployeeID,PostalSuffix
1,122
2,401
3,033
4,052
5,8JR
6,7JR
7,9SP
8,105
9,7LT


## 🗃️ **Mathematical Functions**
Used to perform calculations and operations on numerical data within a SQL query.

## **📍18. Use ROUND to adjust deicimal precison.**

📝 `ROUND` sets the decimal precision of numeric values, helpful for displaying currency or other precise values.  
⚙️ Round product prices to two decimal places.  

🖥️   
SELECT TOP 5 ROUND(UnitPrice, 2) AS RoundedPrice  
FROM Products;  
  
🚫 Could lead to a loss of precision for exact calculations.

In [20]:
SELECT TOP 5 ROUND(UnitPrice, 2) AS RoundedPrice
FROM Products;

(5 rows affected)

Total execution time: 00:00:00.019

RoundedPrice
18.00
19.00
10.00
22.00
21.35


## **📍19. Use CHECKSUM for data integrity checks.**

📝 `CHECKSUM` generates a hash value (acts as a fingerprint) for a row or column, which is useful for detecting changes in data or verifying data integrity.  
⚙️ Track changes in the _Orders_ table by generating `CHECKSUM` values for each row. 

🖥️   
SELECT TOP 5 OrderID,  
    CHECKSUM(*) AS RowChecksum  
FROM Orders;    
  
🚫 `CHECKSUM` values may not be unique for all data, so it’s not always suitable for critical integrity checks.

In [21]:
SELECT TOP 5 OrderID,
	CHECKSUM(*) AS RowChecksum
FROM Orders;

(5 rows affected)

Total execution time: 00:00:00.007

OrderID,RowChecksum
10248,-548360815
10249,423381171
10250,-179432955
10251,-689489561
10252,-337808960


## 🗃️ **Join and Set Operations**
Allow combining data from multiple tables or comparing datasets to find common or distinct values. These operations are essential for creating comprehensive views across related data.

## **📍20. Use EXCEPT to find non-mathcing records.**

📝 `EXCEPT` returns records from the first query that are not in the second query, and can be useful for comparison.  
⚙️ Show customers with no orders. 

🖥️   
SELECT CustomerID  
FROM Customers  
EXCEPT  
SELECT CustomerID  
FROM Orders;    
  
🚫 Not all SQL databases support `EXCEPT`; an alternative could be to use `NOT IN`.

In [24]:
SELECT CustomerID
FROM Customers
EXCEPT
SELECT CustomerID
FROM Orders;

(2 rows affected)

Total execution time: 00:00:00.018

CustomerID
FISSA
PARIS


## **📍21. Use INNER JOIN to combine rows based on related columns.**

📝 `INNER JOIN` combines rows from two tables where there is a match in a related column, useful for pulling together related data.  
⚙️ Retrieve orders with customer names. 

🖥️   
SELECT Top 5 Orders.OrderID,  
    Customers.ContactName  
FROM Orders  
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID  
ORDER BY OrderID;    
  
🚫 `INNER JOIN` only includes rows with matches in both tables; unmatched rows are excluded.

In [26]:
SELECT Top 5 Orders.OrderID,
    Customers.ContactName
FROM Orders
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID
ORDER BY OrderID;

(5 rows affected)

Total execution time: 00:00:00.008

OrderID,ContactName
10248,Paul Henriot
10249,Karin Josephs
10250,Mario Pontes
10251,Mary Saveley
10252,Pascale Cartrain


## **📍22. Use LEFT JOIN to include all rows from the left table.**

📝 `LEFT JOIN` returns all rows from the left table and matched rows from the right table, showing unmatched rows as `NULLS`, which is helpful for inclusive data analysis.  
⚙️ Retrieve orders with customer names. 

🖥️   
SELECT Top 5 Orders.OrderID,  
    Customers.ContactName  
FROM Orders  
INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID  
ORDER BY OrderID;    
  
🚫 `LEFT JOIN` may include nulls for unmatched row.

In [30]:
SELECT TOP 5 Orders.OrderID,
    Shippers.CompanyName
FROM Orders
LEFT JOIN Shippers ON Orders.ShipVia = Shippers.ShipperID;

(5 rows affected)

Total execution time: 00:00:00.016

OrderID,CompanyName
10248,Federal Shipping
10249,Speedy Express
10250,United Package
10251,Speedy Express
10252,United Package


## 🗃️ **Data Type Conversion and Conditional Functions**
Manage data compatibility and apply conditional logic in queries

## **📍23. Use CONVERT to change data types.**

📝 `CONVERT` changes a value from one data type to another, which is essential when combining data of different types or formatting data for display..  
⚙️ Convert the _OrderDate to a different format. 

🖥️     
SELECT TOP 5 OrderID,  
    CONVERT(varchar, OrderDate, 101) AS FormattedOrderDate  
FROM Orders;   
  
Here is a table of `CONVERT` codes for date formats.

| **Code** | **Format**                      | **Example (Output for `2024-10-30`)**      |
|----------|---------------------------------|--------------------------------------------|
| **0**    | Default (mon dd yyyy hh:miAM/PM)| Oct 30 2024 12:00AM                        |
| **1**    | mm/dd/yy                        | 10/30/24                                   |
| **2**    | yy.mm.dd                        | 24.10.30                                   |
| **3**    | dd/mm/yy                        | 30/10/24                                   |
| **4**    | dd.mm.yy                        | 30.10.24                                   |
| **5**    | dd-mm-yy                        | 30-10-24                                   |
| **10**   | mm-dd-yy                        | 10-30-24                                   |
| **11**   | yy/mm/dd                        | 24/10/30                                   |
| **12**   | yy.mm.dd                        | 24.10.30                                   |
| **20**   | yyyy-mm-dd (ISO 8601)           | 2024-10-30                                 |
| **21**   | yyyy-mm-dd hh:mi:ss.mmm (ODBC)  | 2024-10-30 00:00:00.000                    |
| **23**   | yyyy-mm-dd                      | 2024-10-30                                 |
| **101**  | mm/dd/yyyy                      | 10/30/2024                                 |
| **102**  | yyyy.mm.dd                      | 2024.10.30                                 |
| **103**  | dd/mm/yyyy                      | 30/10/2024                                 |
| **104**  | dd.mm.yyyy                      | 30.10.2024                                 |
| **105**  | dd-mm-yyyy                      | 30-10-2024                                 |
| **106**  | dd mon yyyy                     | 30 Oct 2024                                |
| **107**  | mon dd, yyyy                    | Oct 30, 2024                               |
| **108**  | hh:mi:ss                        | 00:00:00                                   |
| **110**  | mm-dd-yyyy                      | 10-30-2024                                 |
| **111**  | yyyy/mm/dd                      | 2024/10/30                                 |
| **112**  | yyyymmdd                        | 20241030                                   |
| **113**  | dd mon yyyy hh:mi:ss:mmm        | 30 Oct 2024 00:00:00:000                   |
| **120**  | yyyy-mm-dd hh:mi:ss (ISO)       | 2024-10-30 00:00:00                        |
| **121**  | yyyy-mm-dd hh:mi:ss.mmm (ISO)   | 2024-10-30 00:00:00.000                    |
| **126**  | yyyy-mm-ddThh:mi:ss.mmm (ISO)   | 2024-10-30T00:00:00.000                    |


In [27]:
SELECT TOP 5 OrderID,
	CONVERT(varchar, OrderDate, 101) AS FormattedOrderDate
FROM Orders;

(5 rows affected)

Total execution time: 00:00:00.008

OrderID,FormattedOrderDate
10248,07/04/2021
10249,07/05/2021
10250,07/08/2021
10251,07/08/2021
10252,07/09/2021


## **📍24. Use CAST to change data types.**

📝 Similarly `CAST` is a function for converting data types with broader compatibilty across databases .  
⚙️ Convert _UnitPrice_ to an integer for simpler calculations. 

🖥️  
SELECT TOP 5 ProductName,  
    CAST(UnitPrice AS int) AS RoundedUnitPrice  
FROM Products; 

In [28]:
SELECT TOP 5 ProductName,
	CAST(UnitPrice AS int) AS RoundedUnitPrice
FROM Products;

(5 rows affected)

Total execution time: 00:00:00.008

ProductName,RoundedUnitPrice
Chai,18
Chang,19
Aniseed Syrup,10
Chef Anton's Cajun Seasoning,22
Chef Anton's Gumbo Mix,21


## **📍25. Use TOP to limit results.**

📝  `TOP` limits the number of rows returned, which is useful for sampling data or improving query performance.  
⚙️ Get the top 5 most expensive products.  

🖥️   
SELECT TOP 5 ProductName,   
    UnitPrice    
FROM Products   
ORDER BY UnitPrice DESC;   

🚫 When `TOP` is used without an `ORDER BY` clause, the query could return different rows each time it's run. It's recommended to pair `TOP` with `ORDER BY` to ensure predictable results.

In [29]:
SELECT TOP 5 ProductName,
	UnitPrice
FROM Products
ORDER BY UnitPrice DESC;

(5 rows affected)

Total execution time: 00:00:00.015

ProductName,UnitPrice
Côte de Blaye,263.50
Thüringer Rostbratwurst,123.79
Mishi Kobe Niku,97.00
Sir Rodney's Marmalade,81.00
Carnarvon Tigers,62.50


_All queries use the Northwind\_2023 database._

_Queries have been modified to return limited results for better readbility._

_All emojis in this project are sourced from [emojicopy.com](https:\emojicopy.com\)_